In [94]:
import tensorflow as tf
import importlib
import os
import loss_functions
import dataprocessor
import model
import pickle
import change_point_detection
from tqdm import tqdm
from heartrate import trace

def reload_custom_libs():
    importlib.reload(loss_functions)
    importlib.reload(dataprocessor)
    importlib.reload(change_point_detection)
    importlib.reload(model)



reload_custom_libs()
from change_point_detection import *
from loss_functions import *
from model import *
from dataprocessor import *

macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126
folder_path = "data"
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")][:10]

In [95]:
# 获得高斯变点片段
# test = [0, 200]
gaussion_process_list = []
# for i in tqdm(range(15)):
#     lines = [test[0] + 200 * i, test[1] + 200 * i]
#     # 处理数据
#     data_list = process_data_list(files, macd_timescales, rtn_timescales, test=lines)
#     print(f"{lines[0]}_{lines[1]}数据处理完成")
#     # 获得断点分割片段数据
#     try:
#         gaussion_process_list = get_segment_list(data_list=data_list)
#     except:
#         continue
#     with open(f'segments/{lines[0]}_{lines[1]}.pkl', 'wb') as f:
#         pickle.dump(gaussion_process_list, f)
#     # 读取数据
#     with open(f'segments/{lines[0]}_{lines[1]}.pkl', 'rb') as f:
#         file = pickle.load(f)
#         gaussion_process_list.extend(file)


data_list = process_data_list(files, macd_timescales, rtn_timescales, first_date=20220101, fill=False)

pkl_files = [f for f in os.listdir("segments") if f.endswith(".pkl")]
for file in pkl_files:
    with open("segments/" + file, 'rb') as file:
        data = pickle.load(file)
        gaussion_process_list.extend(data)

In [96]:
data_list[-2]

,date,close,side_info,rtn_next_day,rtn,macd_8_24,macd_16_28,macd_32_96,sigma,rtn_1,rtn_21,rtn_63,rtn_126,rtn_252


# 模型初始化

In [26]:
reload_custom_libs()
from change_point_detection import *
from loss_functions import *
from model import ModelWrapper
from dataprocessor import *


In [ ]:
# 生成数据： target_set 和 context_set
asset_num, context_num = 10, 20
with open(f'map.pkl', 'rb') as f:
    word_index = pickle.load(f)
target_set, labels, word_index = generate_tensors(data_list, time_steps=timesteps, encoder_type = "one-hot", word_index = word_index, return_map=True)
target_set, context_set, labels = gaussian_data_binder(
    data_list,
    target_set,
    labels,
    map=word_index,
    asset_num=asset_num,
    context_num=context_num,
    gaussion_process_list=gaussion_process_list,
)

# 设置参数
target_std = tf.cast(5e-2, tf.float64)
hidden_dim = 64  # 128
warm_up = 63
features_len = len(macd_timescales) + len(rtn_timescales)

In [34]:
# declare 数据, 初始化数据集
x, s = target_set[0], target_set[-1]
x_c_rtn, x_c, s_c = context_set[0], context_set[0][:, :, :, 1:], context_set[-1]
dataset = tf.data.Dataset.from_tensor_slices((x_c, x_c_rtn, s_c, x, s, labels))

timesteps = x.shape[-2]
features_len = x.shape[-1]
encoding_size = s.shape[-1]
x_shape = (None, asset_num, timesteps, features_len)
s_shape = (None, asset_num, timesteps, encoding_size)

# 初始化模型
xtrend_model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads=4, dropout_rate=0.4)
xtrend_model.build((x_shape, s_shape))

# 训练模型

In [ ]:
model.save(f"final_model.keras")